<a href="https://colab.research.google.com/github/PrasadRecharla/DL_CNNModelOptimization_KerasHyperTuner/blob/main/OptimizeCNNModelWithKerasTunerExample.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Create CNN Model and Optimize it using Keras Tuner

In [6]:
!pip install keras-tuner

     |████████████████████████████████| 71kB 9.2MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.2-cp36-none-any.whl size=78937 sha256=2999fab9b04d4948b381186e9d9f9775da751dd0b83001976f097206f4b2c627
  Stored in directory: /root/.cache/pip/wheels/bb/a1/8a/7c3de0efb3707a1701b36ebbfdbc4e67aedf6d4943a1f463d6
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp36-none-any.whl size=15356 sha256=d5943dd91e945dbc75a375b0ff3782665a2c1edd6b9fd0b1e13b840dec4801de
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables


In [3]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [5]:
print(tf.__version__)

2.3.0


In [7]:
fashion_mnist=keras.datasets.fashion_mnist

In [12]:
(train_images,train_labels),(test_images,test_labels)=fashion_mnist.load_data()

4423680/4422102 [==============================] - 0s 0us/step


In [ ]:
import matplotlib.pyplot as plt
for i in range(3):
  plt.imshow(train_images[i])
  plt.show()


In [25]:
train_images=train_images/255.0
test_images=test_images/255.0

In [26]:
train_images[0].shape

(28, 28)

In [27]:
train_images=train_images.reshape(len(train_images),28,28,1)
test_images=test_images.reshape(len(test_images),28,28,1)

In [28]:
train_images.shape

(60000, 28, 28, 1)

In [29]:
def build_model(hp):  
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')
  ])
  
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
  return model

In [30]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [31]:
tuner_search=RandomSearch(build_model,
                          objective='val_accuracy',
                          max_trials=5,directory='output',project_name="Mnist Fashion")

In [32]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

Trial 5 Complete [00h 00m 23s]
val_accuracy: 0.9070000052452087

Best val_accuracy So Far: 0.9128333330154419
Total elapsed time: 00h 02m 27s
INFO:tensorflow:Oracle triggered exit


In [34]:
model=tuner_search.get_best_models(num_models=1)[0]

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 80)        800       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 22, 22, 32)        64032     
_________________________________________________________________
flatten (Flatten)            (None, 15488)             0         
_________________________________________________________________
dense (Dense)                (None, 96)                1486944   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                970       
Total params: 1,552,746
Trainable params: 1,552,746
Non-trainable params: 0
_________________________________________________________________


In [35]:
model.fit(train_images, train_labels, epochs=10, validation_split=0.1, initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 11s 6ms/step - loss: 0.1377 - accuracy: 0.9490 - val_loss: 0.2439 - val_accuracy: 0.9165
Epoch 5/10
1688/1688 [==============================] - 11s 6ms/step - loss: 0.0955 - accuracy: 0.9646 - val_loss: 0.3018 - val_accuracy: 0.9152
Epoch 6/10
1688/1688 [==============================] - 11s 6ms/step - loss: 0.0678 - accuracy: 0.9751 - val_loss: 0.3229 - val_accuracy: 0.9140
Epoch 7/10
1688/1688 [==============================] - 11s 6ms/step - loss: 0.0512 - accuracy: 0.9811 - val_loss: 0.3881 - val_accuracy: 0.9165
Epoch 8/10
1688/1688 [==============================] - 11s 6ms/step - loss: 0.0353 - accuracy: 0.9871 - val_loss: 0.4737 - val_accuracy: 0.9077
Epoch 9/10
1688/1688 [==============================] - 11s 6ms/step - loss: 0.0313 - accuracy: 0.9885 - val_loss: 0.5301 - val_accuracy: 0.9070
Epoch 10/10
1688/1688 [==============================] - 11s 7ms/step - loss: 0.0259 - accuracy: 0.9908 - val_loss: 0.5552 - val_a